In [1]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    Trainer, 
    TrainingArguments
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import numpy as np


c:\Users\clem9\Desktop\ecole\ing3\NLP\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ============================================
# 1. CHARGEMENT DES DONNÉES
# ============================================
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

# Garder seulement les colonnes nécessaires
train_df = train_df[['text', 'label_numeric']].rename(columns={'label_numeric': 'label'})
test_df = test_df[['text', 'label_numeric']].rename(columns={'label_numeric': 'label'})

print(f"Train: {len(train_df)} articles | Test: {len(test_df)} articles")

Train: 30916 articles | Test: 7730 articles


In [3]:
# ============================================
# 2. CONVERSION EN DATASET HUGGINGFACE
# ============================================
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [4]:
# ============================================
# 3. TOKENIZATION
# ============================================
print("\nTokenization...")
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(
        examples['text'], 
        padding="max_length", 
        truncation=True, 
        max_length=512
    )

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)


Tokenization...


Map: 100%|██████████| 7730/7730 [00:02<00:00, 2646.10 examples/s]


In [5]:
# ============================================
# 4. MODÈLE
# ============================================
print("\nChargement du modèle BERT...")
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, 
    num_labels=2
)

# Détecter si GPU disponible
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device utilisé: {device}")



Chargement du modèle BERT...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Device utilisé: cuda


In [6]:
# ============================================
# 5. MÉTRIQUES D'ÉVALUATION
# ============================================
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='binary'
    )
    acc = accuracy_score(labels, preds)
    
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


In [9]:
# ============================================
# 6. CONFIGURATION DE L'ENTRAÎNEMENT
# ============================================
print("\nConfiguration de l'entraînement...")
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,  # Réduit pour GPU 8GB
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    push_to_hub=False,
    logging_steps=100,
    fp16=True,  # Précision mixte pour accélérer
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)


Configuration de l'entraînement...


In [10]:
# ============================================
# 7. ENTRAÎNEMENT
# ============================================
print("\nEntraînement en cours...")

trainer.train()


Entraînement en cours...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.000000,0.003989,0.999483,0.999528,0.999057,1.000000
2,0.000000,0.001453,0.999871,0.999882,1.000000,0.999764


TrainOutput(global_step=15458, training_loss=0.005121192545072739, metrics={'train_runtime': 3044.7436, 'train_samples_per_second': 20.308, 'train_steps_per_second': 5.077, 'total_flos': 1.626868277501952e+16, 'train_loss': 0.005121192545072739, 'epoch': 2.0})

In [ ]:
# ============================================
# 8. ÉVALUATION FINALE
# ============================================
print("\nÉvaluation finale...")
results = trainer.evaluate()

print("\n" + "="*50)
print("RÉSULTATS FINAUX")
print("="*50)
for key, value in results.items():
    print(f"{key}: {value:.4f}")

# ============================================
# 9. MATRICE DE CONFUSION
# ============================================
print("\nGénération de la matrice de confusion...")
predictions = trainer.predict(test_dataset)
preds = predictions.predictions.argmax(-1)
labels = predictions.label_ids

cm = confusion_matrix(labels, preds)
print("\nMatrice de confusion:")
print("                 Predicted")
print("               Fake  Real")
print(f"Actual Fake    {cm[0][0]:4d}  {cm[0][1]:4d}")
print(f"       Real    {cm[1][0]:4d}  {cm[1][1]:4d}")

# ============================================
# 10. SAUVEGARDE
# ============================================
print("\n Sauvegarde du modèle...")
model.save_pretrained("./baseline_model")
tokenizer.save_pretrained("./baseline_model")

print("\n Modèle sauvegardé dans ./baseline_model/")
print("Entraînement terminé avec succès!")


📊 Évaluation finale...



🎯 RÉSULTATS FINAUX
eval_loss: 0.0015
eval_accuracy: 0.9999
eval_f1: 0.9999
eval_precision: 1.0000
eval_recall: 0.9998
eval_runtime: 62.0637
eval_samples_per_second: 124.5500
eval_steps_per_second: 31.1450
epoch: 2.0000

📈 Génération de la matrice de confusion...

Matrice de confusion:
                 Predicted
               Fake  Real
Actual Fake    3491     0
       Real       1  4238

💾 Sauvegarde du modèle...

✅ Modèle sauvegardé dans ./baseline_model/
🎉 Entraînement terminé avec succès!


In [17]:

# ============================================
# 11. TEST DE PRÉDICTION - VERSION CORRIGÉE
# ============================================
print("\n Test de prédiction sur VRAIS articles...")

def predict_news(text):
    """Prédire si une news est fake ou real"""
    inputs = tokenizer(
        text, 
        return_tensors="pt", 
        truncation=True, 
        max_length=512,
        padding=True
    )
    
    inputs = {k: v.to(device) for k, v in inputs.items()}
    model.to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        pred = torch.argmax(probs).item()
        confidence = probs[0][pred].item()
    
    label = "🔴 FAKE" if pred == 0 else "🟢 REAL"
    return label, confidence

# ============================================
# EXEMPLES DE VRAIS ARTICLES (pas de phrases courtes)
# ============================================
test_examples = [
    # FAKE NEWS (style sensationnaliste)
    """Breaking: Scientists at a secret lab have discovered that 
    eating chocolate every day can cure all types of cancer and 
    make you live forever! The pharmaceutical industry is hiding 
    this information because they want to keep selling expensive 
    treatments. Share this before it gets deleted!""",
    
    # REAL NEWS (style journalistique)
    """PARIS (Reuters) - The Eiffel Tower, one of the world's most 
    iconic landmarks, celebrated its 134th anniversary today. Built 
    in 1889 by engineer Gustave Eiffel, the 324-meter iron structure 
    attracts millions of visitors annually and remains a symbol of 
    French culture and engineering excellence.""",
    
    # FAKE NEWS (complot)
    """Leaked documents reveal that Bill Gates and the WHO are planning 
    to implant microchips in COVID vaccines to control the population. 
    The mainstream media refuses to report this shocking truth. Wake up 
    people! They're tracking your every move and reading your thoughts!""",
    
    # FAKE NEWS (santé)
    """Doctors don't want you to know this simple trick! Drinking lemon 
    water with baking soda cures diabetes in just 3 days. Big Pharma 
    hates this because it costs only $2. Thousands of people have already 
    been cured. Try it now before the government bans this information!""",
]

print("\n" + "="*70)
print(" TESTS DE PRÉDICTION SUR ARTICLES COMPLETS")
print("="*70)

for i, example in enumerate(test_examples, 1):
    label, confidence = predict_news(example)
    
    # Afficher un extrait
    preview = example[:100].replace("\n", " ") + "..."
    print(f"\n{i}. {preview}")
    print(f"   → Prédiction: {label}")
    print(f"   → Confiance: {confidence:.2%}")

# ============================================
# TEST AVEC DES VRAIS ARTICLES DU DATASET
# ============================================
print("\n" + "="*70)
print(" TESTS SUR ARTICLES RÉELS DU DATASET")
print("="*70)

# Prendre quelques exemples du test set
sample_articles = test_df.sample(5, random_state=42)

for idx, row in sample_articles.iterrows():
    label, confidence = predict_news(row['text'])
    true_label = "FAKE" if row['label'] == 0 else "REAL"
    
    # Extraire le titre ou début
    preview = row['text'][:100].replace("\n", " ") + "..."
    
    match = "✅" if label.replace("🔴 ", "").replace("🟢 ", "") == true_label else "❌"
    print(f"\n{match} Vrai: {true_label}")
    print(f"   Article: {preview}")
    print(f"   Prédit: {label} ({confidence:.1%})")






 Test de prédiction sur VRAIS articles...

 TESTS DE PRÉDICTION SUR ARTICLES COMPLETS

1. Breaking: Scientists at a secret lab have discovered that      eating chocolate every day can cure a...
   → Prédiction: 🔴 FAKE
   → Confiance: 100.00%

2. PARIS (Reuters) - The Eiffel Tower, one of the world's most      iconic landmarks, celebrated its 13...
   → Prédiction: 🟢 REAL
   → Confiance: 100.00%

3. Leaked documents reveal that Bill Gates and the WHO are planning      to implant microchips in COVID...
   → Prédiction: 🔴 FAKE
   → Confiance: 100.00%

4. Doctors don't want you to know this simple trick! Drinking lemon      water with baking soda cures d...
   → Prédiction: 🔴 FAKE
   → Confiance: 100.00%

 TESTS SUR ARTICLES RÉELS DU DATASET

✅ Vrai: REAL
   Article: TOKYO (Reuters) - Japanese Prime Minister Shinzo Abe is considering calling a snap election for as e...
   Prédit: 🟢 REAL (100.0%)

✅ Vrai: REAL
   Article: WASHINGTON (Reuters) - President Donald Trump and Democratic leaders

In [14]:
# ============================================
# DIAGNOSTIC DU MODÈLE
# ============================================
print("\n DIAGNOSTIC APPROFONDI...")

# 1. Vérifier les labels du dataset
print("\n Vérification des labels:")
print(f"Label 0 = {test_df[test_df['label'] == 0]['label'].count()} articles")
print(f"Label 1 = {test_df[test_df['label'] == 1]['label'].count()} articles")
print("\nRappel: Label 0 = FAKE, Label 1 = REAL")

# 2. Tester sur des exemples du test set réel
print("\n Test sur exemples réels du dataset:")

# Prendre 5 fake et 5 real du test set
fake_samples = test_df[test_df['label'] == 0].sample(5)
real_samples = test_df[test_df['label'] == 1].sample(5)

print("\n FAKE NEWS du dataset:")
for idx, row in fake_samples.iterrows():
    label, conf = predict_news(row['text'])
    true_label = "FAKE" if row['label'] == 0 else "REAL"
    match = "✅" if label.replace("🔴 ", "").replace("🟢 ", "") == true_label else "❌"
    print(f"{match} Vrai: {true_label} | Prédit: {label} ({conf:.1%})")

print("\nREAL NEWS du dataset:")
for idx, row in real_samples.iterrows():
    label, conf = predict_news(row['text'])
    true_label = "FAKE" if row['label'] == 0 else "REAL"
    match = "✅" if label.replace("🔴 ", "").replace("🟢 ", "") == true_label else "❌"
    print(f"{match} Vrai: {true_label} | Prédit: {label} ({conf:.1%})")

# 3. Vérifier les logits bruts
print("\n Vérification des logits bruts:")
test_text = "The Eiffel Tower is in Paris, France."
inputs = tokenizer(test_text, return_tensors="pt", truncation=True, max_length=512)
inputs = {k: v.to(device) for k, v in inputs.items()}

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    probs = torch.nn.functional.softmax(logits, dim=-1)
    
    print(f"Logits bruts: {logits}")
    print(f"Probabilités: Fake={probs[0][0]:.4f}, Real={probs[0][1]:.4f}")
    print(f"Prédiction: {torch.argmax(probs).item()} (0=Fake, 1=Real)")

# 4. Statistiques globales sur tout le test set
print("\n Statistiques sur TOUT le test set (échantillon de 100):")
sample_test = test_df.sample(100, random_state=42)
predictions = []
true_labels = []

for idx, row in sample_test.iterrows():
    label, conf = predict_news(row['text'])
    pred = 0 if "FAKE" in label else 1
    predictions.append(pred)
    true_labels.append(row['label'])

from sklearn.metrics import confusion_matrix, accuracy_score

acc = accuracy_score(true_labels, predictions)
cm = confusion_matrix(true_labels, predictions)

print(f"\nAccuracy sur 100 exemples: {acc:.2%}")
print("\nMatrice de confusion:")
print(f"              Predicted")
print(f"            Fake  Real")
print(f"Actual Fake {cm[0][0]:4d}  {cm[0][1]:4d}")
print(f"       Real {cm[1][0]:4d}  {cm[1][1]:4d}")

print(f"\n Distribution des prédictions:")
fake_pred = sum(1 for p in predictions if p == 0)
real_pred = sum(1 for p in predictions if p == 1)
print(f"Prédit FAKE: {fake_pred}/100 ({fake_pred}%)")
print(f"Prédit REAL: {real_pred}/100 ({real_pred}%)")


 DIAGNOSTIC APPROFONDI...

 Vérification des labels:
Label 0 = 3491 articles
Label 1 = 4239 articles

Rappel: Label 0 = FAKE, Label 1 = REAL

 Test sur exemples réels du dataset:

 FAKE NEWS du dataset:
✅ Vrai: FAKE | Prédit: 🔴 FAKE (100.0%)
✅ Vrai: FAKE | Prédit: 🔴 FAKE (100.0%)
✅ Vrai: FAKE | Prédit: 🔴 FAKE (100.0%)
✅ Vrai: FAKE | Prédit: 🔴 FAKE (100.0%)
✅ Vrai: FAKE | Prédit: 🔴 FAKE (100.0%)

REAL NEWS du dataset:
✅ Vrai: REAL | Prédit: 🟢 REAL (100.0%)
✅ Vrai: REAL | Prédit: 🟢 REAL (100.0%)
✅ Vrai: REAL | Prédit: 🟢 REAL (100.0%)
✅ Vrai: REAL | Prédit: 🟢 REAL (100.0%)
✅ Vrai: REAL | Prédit: 🟢 REAL (100.0%)

 Vérification des logits bruts:
Logits bruts: tensor([[ 5.6953, -5.2773]], device='cuda:0')
Probabilités: Fake=1.0000, Real=0.0000
Prédiction: 0 (0=Fake, 1=Real)

 Statistiques sur TOUT le test set (échantillon de 100):

Accuracy sur 100 exemples: 100.00%

Matrice de confusion:
              Predicted
            Fake  Real
Actual Fake   51     0
       Real    0    49

 Distribu